In [719]:

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from pathlib import Path 
import sys 
import os 
import logging
import seaborn as sns

from scipy.stats import uniform, randint
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, GroupShuffleSplit

from collections import Counter
from sklearn.preprocessing import FunctionTransformer, StandardScaler

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GroupKFold,  cross_validate
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor 
from lightgbm import LGBMRegressor

from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from skrub import TableVectorizer


from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, classification_report, confusion_matrix

sys.path.append('/Users/alina/Desktop/MIT/code/ADHD/MTA/helper')
from helper import rr, prep, var_dict, audit, plot, save


%load_ext autoreload
%autoreload 2

#shap.initjs()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [720]:
if Path('/Volumes/Samsung_T5/MIT/mta').exists():
    data_root =     '/Volumes/Samsung_T5/MIT/mta'
    data_derived  = '/Volumes/Samsung_T5/MIT/mta/output/derived_data'
else: 
    data_root = '/Users/alina/Desktop/MIT/code/data'
    data_derived  = '/Users/alina/Desktop/MIT/code/data/output/derived_data'

info_path = Path(data_root, "files")
#os.listdir(data_derived)


In [721]:
################## PREDICTORS ####################
pred = pd.read_csv(Path(data_derived, 'mta_data_clean.csv')).drop(columns = 'Unnamed: 0')
out = pd.read_csv(Path(data_derived, 'out_clean_all_raters.csv')).drop(columns = 'Unnamed: 0')

rater_out = 'm'
rater_pred = None # if set to None use all raters 

thr_drop_missing = 50 # if changed to lower number, some cols will be dropped, 50 iwas threshold in preprcessing (wont change anything)


In [722]:

if rater_pred is not None:
    col_pred = [ col for col in pred.columns if col.endswith(rater_pred)]
    col_pred.append("src_subject_id")
    col_pred.append('trtname')
    pred = pred[col_pred]
    
    
 ################ OUTCOMES ##################   

col_out_rater = [col for col in out if col.endswith(rater_out)]

out_rater = out[np.concatenate((['src_subject_id'], col_out_rater))]

for col in col_out_rater:
    print(col, out_rater[col].dropna().shape)

out_rater = out_rater.rename(columns={col: f"{col}_out" for col in out_rater.columns if col != 'src_subject_id'})


################ MERGE ##############
data = pd.merge(out_rater, pred, how = 'left', on = 'src_subject_id')
data = audit.remove_cols(data, thr_drop_missing=thr_drop_missing) # adjust if desired, as loaded thr was 50 

y_cols = [col for col in data.columns if col.endswith('out')]
print('possible outcomes : ', y_cols)
y_col = y_cols[2] # extrcat single outcome to predict 
print('To predict : ', y_col)

X_cols = [col for col in data.columns if col not in y_cols]
data = data.dropna(subset=[y_col])

y = np.array(data[y_col])
df_X = data[X_cols].drop(columns='src_subject_id')

snap_snainatt_m (502,)
snap_snahypat_m (503,)
snap_snaoddt_m (503,)
ssrs_sspintt_m (497,)
ssrs_ssptosst_m (496,)
pcrc_pcrcpax_m (520,)
pcrc_pcrcprx_m (521,)
Index([], dtype='object')
Removing empty columns ..  N =  0
Index([], dtype='object')
(559, 720)
Removing constant columns .. N =  0
Index([], dtype='object')
(559, 720)
Removing known and raw columns..  N =  : 0
[]
(559, 720)
Removing above threshold empty columns.. N =  : 0
set()
(559, 720)
possible outcomes :  ['snap_snainatt_m_out', 'snap_snahypat_m_out', 'snap_snaoddt_m_out', 'ssrs_sspintt_m_out', 'ssrs_ssptosst_m_out', 'pcrc_pcrcpax_m_out', 'pcrc_pcrcprx_m_out']
To predict :  snap_snaoddt_m_out


In [723]:

col_names_data = list(data.columns)

ord_vars, num_vars, cat_vars = [], [], []

types_df = pd.read_excel(Path(data_derived, 'all_vars_description_ML.xlsx'), sheet_name='Sheet1')

for _, row in types_df.iterrows():
    var_name = row[1]  # e.g. variable name in the spreadsheet
    var_type = row[4]  # e.g. "ord" / "num" / "cat"

    # Collect all columns in `data` that contain `var_name`
    var_in_data = [col for col in col_names_data if var_name in col]

    if var_type == "ord":
        ord_vars.append(var_in_data)
    elif var_type == "num":
        num_vars.append(var_in_data)
    elif var_type == "cat":
        cat_vars.append(var_in_data)

# Example: manually add a column named 'trtname' to cat_vars
cat_vars.append(['trtname'])

# Flatten each list-of-lists into a single array
ord_vars = np.concatenate(ord_vars)
cat_vars = np.concatenate(cat_vars)
num_vars = np.concatenate(num_vars)

# Convert them to plain Python strings
ord_vars = [str(col) for col in ord_vars]
cat_vars = [str(col) for col in cat_vars]
num_vars = [str(col) for col in num_vars]

print("Ordinal vars:", ord_vars)
print("Numeric vars:", num_vars)
print("Categorical vars:", cat_vars)

num_vars_in = [str(col) for col in num_vars if not col.endswith("out")] # name of numerical variables present in dataframe X (excludin var names in y)


Ordinal vars: ['snap_sna19_m', 'snap_sna19_f', 'snap_sna19_t', 'snap_sna20_m', 'snap_sna20_f', 'snap_sna20_t', 'snap_sna21_m', 'snap_sna21_f', 'snap_sna21_t', 'snap_sna22_m', 'snap_sna22_f', 'snap_sna22_t', 'snap_sna23_m', 'snap_sna23_f', 'snap_sna23_t', 'snap_sna24_m', 'snap_sna24_f', 'snap_sna24_t', 'snap_sna25_m', 'snap_sna25_f', 'snap_sna25_t', 'snap_sna26_m', 'snap_sna26_f', 'snap_sna26_t', 'snap_sna27_m', 'snap_sna27_f', 'snap_sna27_t', 'snap_sna28_m', 'snap_sna28_f', 'snap_sna28_t', 'snap_sna29_m', 'snap_sna29_f', 'snap_sna29_t', 'snap_sna30_m', 'snap_sna30_f', 'snap_sna30_t', 'snap_sna31_m', 'snap_sna31_f', 'snap_sna31_t', 'snap_sna32_m', 'snap_sna32_f', 'snap_sna32_t', 'snap_sna33_m', 'snap_sna33_f', 'snap_sna33_t', 'snap_sna34_m', 'snap_sna34_f', 'snap_sna34_t', 'snap_sna35_m', 'snap_sna35_f', 'snap_sna35_t', 'snap_sna36_m', 'snap_sna36_f', 'snap_sna36_t', 'snap_sna37_m', 'snap_sna37_f', 'snap_sna37_t', 'snap_sna38_m', 'snap_sna38_f', 'snap_sna38_t', 'snap_sna39_m', 'snap_sna

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_57889/2697639973.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  var_name = row[1]  # e.g. variable name in the spreadsheet
/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_57889/2697639973.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  var_type = row[4]  # e.g. "ord" / "num" / "cat"


In [724]:
num_set = set(num_vars_in)
ord_set = set(ord_vars)
cat_set = set(cat_vars)

# Pairwise overlaps:
overlap_num_ord = num_set & ord_set
overlap_num_cat = num_set & cat_set
overlap_ord_cat = ord_set & cat_set

print("Overlaps between numeric and ordinal:", overlap_num_ord)
print("Overlaps between numeric and categorical:", overlap_num_cat)
print("Overlaps between ordinal and categorical:", overlap_ord_cat)

# Overlap across all three:
overlap_all_three = num_set & ord_set & cat_set
print("Overlaps across numeric, ordinal and categorical:", overlap_all_three)

Overlaps between numeric and ordinal: {'masc_ma22acx_c', 'masc_ma31hfx_c'}
Overlaps between numeric and categorical: set()
Overlaps between ordinal and categorical: set()
Overlaps across numeric, ordinal and categorical: set()


In [725]:

cat_vars_str, cat_vars_num, rest = [], [], []

for col in cat_vars:
    # Get the first non-null value in the column
    val = data[col].dropna().unique()[0]
    
    # Check if it's a (Python or NumPy) string
    if isinstance(val, (str, np.str_)):
        cat_vars_str.append(str(col))  # ensure column name is a Python str
    # Check if it's a (Python or NumPy) float
    elif isinstance(val, (float, np.floating)):
        cat_vars_num.append(str(col))  # ensure column name is a Python str
    else:
        rest.append(str(col))          # store in `rest` for debugging
print(cat_vars_str)
print(cat_vars_num)

['scid_ethnic_group_m', 'scid_das1ms_m', 'trtname']
['scid_axisidx_m', 'scid_axisidx_f', 'scid_pastpsy_m', 'scid_maniachk_m', 'scid_dyschk_m', 'scid_mochk1_m', 'scid_sualchk_m', 'scid_smchk1_m', 'scid_smchk2_m', 'scid_adjchk_m', 'cbcl_special_ed_m', 'cbcl_special_ed_f', 'cbcl_repeat_grade_m', 'cbcl_repeat_grade_f', 'cbcl_academic_m', 'cbcl_academic_f']


In [726]:
# Define the extensions to check
extensions = ['_m', #mother 
              #'_p'# proffesionals
              '_f', # father,
              '_c',# child,
              '_t'] # teacher 

# Count columns for each extension
extension_counts = {ext: sum(col.endswith(ext) for col in pred.columns) for ext in extensions}

for ext, count in extension_counts.items():
    print(f"Columns ending with '{ext}': {count}")

Columns ending with '_m': 277
Columns ending with '_f': 228
Columns ending with '_c': 68
Columns ending with '_t': 138


In [727]:
params = { 'tune_hyperparam' : False,
          'temp_size' : 0.3, # Size of 
          'test_size' : 0.3,
          'random_state' : 42, 
          'num_top_features': 10, # number of features to keep after featur selection, change to when it doesnt get better anymore 
          'stratify' : 'trtname',
          'gkf_n_splits' : 5} # in balance in train teat split 


scoring = {
    'r2': 'r2',
    'mse': 'neg_mean_squared_error',
    'mae': 'neg_mean_absolute_error'
}

groups = data['src_subject_id'].values

In [728]:
# ----------------------------
# Custom Transformers
# ----------------------------

class CorrelationSelector(BaseEstimator, TransformerMixin):
    """
    Selects features based on a correlation threshold.
    Removes features that have a correlation higher than the specified threshold with any other feature.
    """
    def __init__(self, threshold=0.8):
        self.threshold = threshold
        self.to_drop_ = None
        self.features_to_keep_ = None

    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise ValueError("CorrelationSelector expects input X to be a pandas DataFrame")
        
        corr_matrix = X.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        self.to_drop_ = [column for column in upper.columns if any(upper[column] > self.threshold)]
        self.features_to_keep_ = [column for column in X.columns if column not in self.to_drop_]
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise ValueError("CorrelationSelector expects input X to be a pandas DataFrame")
        return X[self.features_to_keep_]

    def get_support(self, indices=False):
        if indices:
            return [i for i, col in enumerate(self.features_to_keep_)]
        else:
            return self.features_to_keep_

In [729]:
df_X.columns.duplicated()

# To list just the duplicated columns:
dup_cols = df_X.columns[df_X.columns.duplicated()]
print("Duplicated column names:", dup_cols.tolist())

Duplicated column names: []


In [730]:
df_X.columns.duplicated()

# To list just the duplicated columns:
dup_cols = df_X.columns[df_X.columns.duplicated()]
print("Duplicated column names:", dup_cols.tolist())
X_select = df_X[list(set(num_vars_in + ord_vars + cat_vars))]
X_select.columns.duplicated()

# To list just the duplicated columns:
dup_cols = X_select.columns[X_select.columns.duplicated()]
print("Duplicated column names:", dup_cols.tolist())

Duplicated column names: []
Duplicated column names: []


In [731]:
X_select.columns.duplicated()

# To list just the duplicated columns:
dup_cols = X_select.columns[X_select.columns.duplicated()]
print("Duplicated column names:", dup_cols.tolist())

Duplicated column names: []


In [732]:
for col in X_select:
    print(X_select[col].isna().sum())

198
29
206
21
194
196
194
4
11
10
212
128
124
10
14
123
21
23
27
19
0
241
9
197
7
7
9
33
195
197
14
8
218
210
31
5
33
220
204
5
12
21
20
5
123
197
211
125
123
194
194
28
5
195
3
210
11
14
212
124
29
31
10
196
3
200
5
31
3
10
36
31
7
11
20
33
197
12
10
30
206
211
196
35
19
239
7
31
126
196
206
3
211
19
7
7
32
30
30
21
27
30
192
32
31
125
211
13
8
196
211
9
10
11
21
195
16
32
197
6
128
6
199
6
10
11
7
36
196
127
23
4
197
5
28
203
22
27
5
11
198
8
198
128
123
124
199
124
211
122
124
29
38
122
13
225
7
60
15
27
6
11
124
28
29
196
15
11
202
200
38
34
34
125
35
194
30
23
11
198
210
18
34
25
195
20
197
197
124
9
34
197
27
27
13
9
28
19
12
5
197
16
198
199
198
31
10
30
30
126
141
196
205
19
4
195
124
92
5
21
27
125
31
125
9
19
20
196
9
27
29
124
10
7
3
28
5
200
31
33
28
8
123
125
11
0
19
25
210
30
30
38
7
198
13
6
21
8
126
196
6
21
208
11
26
123
31
10
13
125
202
195
199
125
195
4
24
20
202
27
4
123
201
8
197
195
34
195
8
29
124
12
19
33
36
3
11
197
196
215
126
194
199
8
8
35
2
6
20
124
202
23


In [733]:
from sklearn.svm import SVR

In [734]:
# Ensure X is a DataFrame
X = X_select.copy()

# Initialize GroupShuffleSplit for initial train-test split
gss_test = GroupShuffleSplit(n_splits=1, test_size=params['test_size'], random_state=params['random_state'])
train_val_idx, test_idx = next(gss_test.split(X, y, groups=groups))

# Split data into training-validation and test sets
X_test, y_test, groups_test = X.iloc[test_idx], y[test_idx], groups[test_idx]
X_train_val, y_train_val, groups_train_val = X.iloc[train_val_idx], y[train_val_idx], groups[train_val_idx]

print(f"Total samples: {X.shape[0]}")
print('TRAIN_VAL ', X_train_val.shape, y_train_val.shape)
print('TEST', X_test.shape, y_test.shape)

Total samples: 503
TRAIN_VAL  (352, 712) (352,)
TEST (151, 712) (151,)


In [735]:
def to_dataframe(X):
    # Create a DataFrame with generic column names: f0, f1, f2, ...
    n_cols = X.shape[1]
    return pd.DataFrame(X, columns=[f"f{j}" for j in range(n_cols)])
to_df_transformer = FunctionTransformer(func=to_dataframe, validate=False)

In [738]:
regress =  RandomForestRegressor(random_state=42)


num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('std_scaler', StandardScaler())
])

ord_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('identity', FunctionTransformer(lambda x: x))
])


cat_to_dataframe = FunctionTransformer(
    func=lambda X: pd.DataFrame(X, columns=cat_vars_str),
    validate=False  # prevent extra validation that may alter the input
)

cat_str_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('to_df', cat_to_dataframe),        # conversion step added
     ('ohe', TableVectorizer())
])

cat_num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=-1)),
    ('ohe', OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ('num', num_pipe, num_vars_in),
    ('cat_str', cat_str_pipe, cat_vars_str),
    ('cat_num', cat_num_pipe, cat_vars_num),
    ('ord', ord_pipe, ord_vars)
])


pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
   # ('to_df', to_df_transformer),
   # ('correlation_selector', CorrelationSelector(threshold=0.8)),
    ('regressor', regress)
])

pipeline.fit(X_train_val, y_train_val)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('std_scaler',
                                                                   StandardScaler())]),
                                                  ['snap_snainatt_m',
                                                   'snap_snainatt_f',
                                                   'snap_snainatt_t',
                                                   'snap_snahypat_m',
                                                   'snap_snahypat_f',
                                                   'snap_snahypat_t',
                                                   'snap_snaimput_m',
                                                   'snap_snaimput_f',
                                                   'snap_snaimput_t',
                                                   'snap_snaaddwt_m',
                                                   'snap_snaaddwt_f',
                                                   's...
                                                   'snap_sna22_f',
                                                   'snap_sna22_t',
                                                   'snap_sna23_m',
                                                   'snap_sna23_f',
                                                   'snap_sna23_t',
                                                   'snap_sna24_m',
                                                   'snap_sna24_f',
                                                   'snap_sna24_t',
                                                   'snap_sna25_m',
                                                   'snap_sna25_f',
                                                   'snap_sna25_t',
                                                   'snap_sna26_m',
                                                   'snap_sna26_f',
                                                   'snap_sna26_t',
                                                   'snap_sna27_m',
                                                   'snap_sna27_f',
                                                   'snap_sna27_t',
                                                   'snap_sna28_m',
                                                   'snap_sna28_f',
                                                   'snap_sna28_t', ...])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [739]:
y_pred = pipeline.predict(X_test)
# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 20.595693798079473
Root Mean Squared Error: 4.538247877549162
Mean Absolute Error: 3.485480132450331
R^2 Score: 0.3365229597903191


In [740]:
print(y_col)
X_sample = X_train_val 
X_processed = pipeline[:-1].transform(X_sample)
num_features = X_processed.shape[1]
print("Number of features after processing:", num_features)
print(pipeline.named_steps['regressor'].get_params())

snap_snaoddt_m_out
Number of features after processing: 1021
{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [741]:
pipeline.fit(X_train_val, y_train_val)
predictions = pipeline.predict(X_test)
#print(predictions)


In [ ]:
pipeline.named_steps['regressor'].get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}